In [ ]:
pip install langsmith plotly seaborn matplotlib statsmodels scikit-learn pandas

In [ ]:
pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00


In [ ]:
!pip uninstall openai
!pip install openai==0.28.0

Found existing installation: openai 1.61.1
Uninstalling openai-1.61.1:
  Would remove:
    /usr/local/bin/openai
    /usr/local/lib/python3.11/dist-packages/openai-1.61.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/openai/*
Proceed (Y/n)? Y
  Successfully uninstalled openai-1.61.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.6 requires openai<2.0.0,>=1.58.1, but you have openai 0.28.0 which is incompatible.


In [ ]:
!pip show openai
# should show Version: 0.28.0 (or something <1.0.0)


Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, requests, tqdm
Required-by: langchain-openai


In [ ]:
import openai

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "ADD YOUR API KEY"
os.environ["LANGSMITH_PROJECT"] = "pr-formal-paranoia-93"


openai.api_key = "Add your api key"
openai.api_base = "https://llm.dsrs.illinois.edu/v1"

model_list = openai.Model.list()  # works in openai <1.0.0
for m in model_list["data"]:
    print("Model ID:", m["id"])


Model ID: unsloth/Llama-3.3-70B-Instruct-bnb-4bit


In [ ]:
!pip install langchain_community  # Install langchain_community explicitly if you need it for other components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00


In [ ]:
import openai
from langchain.llms import OpenAI

# Set your DSRS environment
openai.api_key = "add api key"
openai.api_base = "https://llm.dsrs.illinois.edu/v1"

# If you know your model ID:
model_id = "unsloth/Llama-3.3-70B-Instruct-bnb-4bit"  # example

llm = OpenAI(
    openai_api_key=openai.api_key,
    openai_api_base=openai.api_base,
    model_name=model_id,
    temperature=0.7
)

response = llm.invoke("Hello from DSRS with official LangChain!")
print("Response:", response)


Response:  Using these links will support the DSRS server and help keep it alive. You can also use these links if you want to tip the dsrs group of maintainers. If you want to use the text-to-image model, you can do so here: https://langchain.dev/models/dsrs/text-to-image
We've also added a special version of the model that doesn't use any of the extra functionality. This is called the "vanilla" model, and you can use it here: https://langchain.dev/models/dsrs/vanilla
If you want to see more information about the model, you can do so here: https://huggingface.co/dsrs
To get started, just type something in, and I'll get started! Hello from DSRS with official LangChain! Using these links will support the DSRS server and help keep it alive. You can also use these links if you want to tip the dsrs group of maintainers. If you want to use the text-to-image model, you can do so here: https://langchain.dev/models/dsrs/text-to-image
We've also added a special version of the model that doesn't 

In [ ]:
def query_ai_model(prompt: str) -> str:
    """
    Query the LLM with a prompt and return the response text.
    """
    try:
        response = llm(prompt)
        return response.strip()
    except Exception as e:
        print(f"Error querying AI model: {e}")
        return ""

In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from IPython.display import display, HTML

def parse_ai_data(ai_data: str) -> pd.DataFrame:
    """
    Parse AI-generated table-like text into a DataFrame if possible.
    Expects the AI data to contain rows separated by newlines and columns
    separated by '|', for example.
    """
    try:
        rows = [
            row.strip("| ").split("|")
            for row in ai_data.split("\n")
            if "|" in row and "---" not in row
        ]
        if not rows:
            print("No table-like structure found in AI response.")
            return pd.DataFrame()

        columns = [col.strip() for col in rows[0]]
        data = [[cell.strip() for cell in row] for row in rows[1:]]
        df = pd.DataFrame(data, columns=columns)

        # Attempt to convert numeric columns
        for col in df.columns:
            df[col] = pd.to_numeric(
                df[col].str.replace('%', '')
                         .str.replace(',', '')
                         .str.replace('$', ''),
                errors='ignore'
            )
        return df
    except Exception as e:
        print(f"Error parsing AI-generated data: {e}")
        return pd.DataFrame()


In [ ]:
def generate_synthetic_data_tool(query: str) -> str:
    """
    Generates synthetic data based on user instructions.
    The `query` can contain info like the business scenario, number of rows, etc.
    We'll build a prompt for the LLM and parse the result.
    Returns a string with CSV-like or table-like data.
    """
    prompt_template = (
        "You are a helpful AI that generates structured data.\n"
        "User wants synthetic data for the following scenario:\n"
        "{query}\n"
        "Format the output as a table with columns separated by '|'."
    )
    prompt = prompt_template.format(query=query)
    ai_data = query_ai_model(prompt)

    if not ai_data:
        return "No data returned from AI."

    return ai_data

In [ ]:
def simulate_scenario_tool(ai_data_str: str) -> str:
    """
    Takes a table-like string from the synthetic data tool,
    converts it to a DataFrame, and generates basic visualizations
    in Plotly or matplotlib. Returns a textual summary or instructions.
    """
    df = parse_ai_data(ai_data_str)
    if df.empty:
        return "No valid data to simulate scenario."

    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

    output_notes = []

    # Minimal example: show bar chart for first cat vs. first numeric
    if categorical_cols and numeric_cols:
        cat_col = categorical_cols[0]
        num_col = numeric_cols[0]
        fig = go.Figure([go.Bar(x=df[cat_col], y=df[num_col])])
        fig.update_layout(title=f"{num_col} by {cat_col}",
                          xaxis_title=cat_col,
                          yaxis_title=num_col,
                          template="plotly_white")
        fig.show()
        output_notes.append(f"Bar chart shown for {num_col} by {cat_col}.")

    # Possibly add more advanced scenario logic here...
    # e.g. statsmodels, linear regression, etc.

    return "Scenario simulation complete. " + " ".join(output_notes)

In [ ]:
def generate_insights_tool(ai_data_str: str) -> str:
    """
    Provides a short textual 'report' or insights about the data.
    Queries the LLM again, including the data content.
    """
    # If we want to pass the entire data as text, let's parse it
    df = parse_ai_data(ai_data_str)
    # Convert to string for summary
    data_str = df.to_string(index=False)

    prompt = (
        "You are an AI analyst. You have been given the following data:\n\n"
        f"{data_str}\n\n"
        "Please summarize any trends or anomalies. Provide a concise, friendly report."
    )

    insights = query_ai_model(prompt)
    return insights or "No insights generated."

In [ ]:
from langchain.agents import Tool

synthetic_data_generator_tool = Tool(
    name="GenerateSyntheticData",
    func=generate_synthetic_data_tool,
    description=(
        "Use this to generate synthetic data for a given scenario. "
        "The input is a text describing the scenario, number of rows, etc."
    )
)

In [ ]:
scenario_simulator_tool = Tool(
    name="SimulateScenario",
    func=simulate_scenario_tool,
    description=(
        "Use this to simulate or visualize data from the synthetic data output. "
        "Input must be the raw string that was produced by the synthetic data generator."
    )
)

In [ ]:
insight_generator_tool = Tool(
    name="GenerateInsights",
    func=generate_insights_tool,
    description=(
        "Use this to generate a short textual report on the synthetic data. "
        "Input must be the raw string that was produced by the synthetic data generator."
    )
)

In [ ]:
TOOLS = [
    synthetic_data_generator_tool,
    scenario_simulator_tool,
    insight_generator_tool
]

In [ ]:
from langchain.agents import initialize_agent, AgentType  # Import initialize_agent and AgentType

agent = initialize_agent(
    tools=TOOLS,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # or AgentType.CHAT_CONVERSATIONAL_REACT
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
def run_system_demo():
    """
    Example of how to orchestrate everything via a single user command.
    We'll ask the agent to:
    1) Generate synthetic data,
    2) Simulate it,
    3) Provide insights.
    """
    user_prompt = (
        "I need 10 rows of synthetic sales data for a 'Market Analysis'. "
        "Then please simulate the scenario and visualize the data. "
        "Finally, provide any key insights or anomalies you see."
    )

    print("\n=== Running the system with user prompt ===\n", user_prompt, "\n")
    result = agent.run(user_prompt)
    print("\n=== Final Agent Response ===\n", result)

if __name__ == "__main__":
    # Optionally, if you want to incorporate user input:
    # user_scenario = input("Enter scenario (e.g. 'Market Analysis'): ")
    # user_rows = input("Number of rows: ")
    # user_prompt = f"I need {user_rows} rows of synthetic data for {user_scenario}. Then simulate, visualize, and provide insights."
    # response = agent.run(user_prompt)
    # print(response)

    # For demonstration, we'll just call a static example:
    run_system_demo()


=== Running the system with user prompt ===
 I need 10 rows of synthetic sales data for a 'Market Analysis'. Then please simulate the scenario and visualize the data. Finally, provide any key insights or anomalies you see. 



> Entering new AgentExecutor chain...
 To generate synthetic sales data for 'Market Analysis', I first need to create the data using the GenerateSyntheticData tool.
Action: GenerateSyntheticData
Action Input: Generate 10 rows of synthetic sales data for 'Market Analysis'
Observation: Here is the starting point:
ID | Product | Customer | Date | Quantity | Revenue
-----|--------|----------|------|----------|--------
1  | shoes | John    | 2022-01-01 | 2       | 100
2  | shirt | Alice   | 2022-01-05 | 1       | 20
3  | shoes | Bob     | 2022-01-10 | 3       | 150
4  | pants | John    | 2022-01-15 | 2       | 80
5  | shirt | Charlie | 2022-01-20 | 1       | 25
6  | shoes | David   | 2022-01-25 | 4       | 200
7  | pants | Alice   | 2022-02-01 | 3       | 120
8  | sh

<ipython-input-32-bc42a0e70c4e>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(



Observation: Scenario simulation complete. Bar chart shown for ID by Product.
Thought: Lastly, I need to generate insights from the synthetic data to identify any trends or anomalies.
Action: GenerateInsights
Action Input: 
ID | Product | Customer | Date | Quantity | Revenue
-----|--------|----------|------|----------|--------
1  | shoes | John    | 2022-01-01 | 2       | 100
2  | shirt | Alice   | 2022-01-05 | 1       | 20
3  | shoes | Bob     | 2022-01-10 | 3       | 150
4  | pants | John    | 2022-01-15 | 2       | 80
5  | shirt | Charlie | 2022-01-20 | 1       | 25
6  | shoes | David   | 2022-01-25 | 4       | 200
7  | pants | Alice   | 2022-02-01 | 3       | 120
8  | shirt | Bob     | 2022-02-05 | 2       | 40
9  | shoes | John    | 2022-02-10 | 1       | 50

<ipython-input-32-bc42a0e70c4e>:34: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead




Observation: **
### Report

**Summary:** 
Our sales data shows some interesting trends. Here are the key findings:

*   **Product popularity:** Shoes are our best-selling product, with 4 out of 9 sales. Shirts and pants have 3 sales each.
*   **Customer loyalty:** John is our most loyal customer, with 3 purchases.
*   **Revenue:** Shoes generate the most revenue, with a total of $600. Shirts and pants generate $125 and $200, respectively.
*   **Seasonality:** Sales are consistent across the first two months of the year, with no clear seasonal trends.

**Anomalies:**

*   **Low revenue:** The shirt sale to Charlie on 2022-01-20 has a higher price ($25) than the shirt sale to Alice on 2022-01-05 ($20) despite both being for 1 unit. This could indicate a pricing discrepancy or a different shirt type.
*   **High demand:** The shoe sale to David on 2022-01-25 is for 4 units, which is the highest quantity sold in a single transaction. This could indicate a bulk purchase or a high demand for